In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
train_dir = r'E:/other_data/overhead/training' + "//"
test_dir = r'E:/other_data/overhead/testing' + "//"

In [3]:
classes=pd.read_csv('E:/other_data/overhead/classes.csv')
classes

,class,label,train_count,test_count
0,car,0,888,112
1,harbor,1,889,111
2,helicopter,2,655,82
3,oil_gas_field,3,888,110
4,parking_lot,4,888,112
5,plane,5,889,111
6,runway_mark,6,801,100
7,ship,7,889,111
8,stadium,8,843,105
9,storage_tank,9,889,111


In [4]:
Name=classes['class']
Label=classes['label']

file_reverse_mapping=dict(zip(Label,Name)) 
print(file_reverse_mapping)

{0: 'car', 1: 'harbor', 2: 'helicopter', 3: 'oil_gas_field', 4: 'parking_lot', 5: 'plane', 6: 'runway_mark', 7: 'ship', 8: 'stadium', 9: 'storage_tank'}


In [5]:
first_pic_dir = os.path.join(train_dir,Name[0])
file_name = os.listdir(first_pic_dir)[0]
file_full_path = os.path.join(first_pic_dir,file_name)
print("file_full_path :",file_full_path)
pb_test_pic=load_img(file_full_path, grayscale=False, color_mode='rgb', target_size=(48,48))
pb_test_pic=img_to_array(pb_test_pic)
pb_test_pic=pb_test_pic/255.0

pb_test_pic = np.array(pb_test_pic)
pb_test_pic = pb_test_pic.reshape(tuple([1])+pb_test_pic.shape)
print(pb_test_pic.shape)


file_full_path : E:/other_data/overhead/training//car\00003.jpg
(1, 48, 48, 3)


In [6]:
print(type(pb_test_pic))

<class 'numpy.ndarray'>


In [7]:
# pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(48,48,3),include_top=False,weights='imagenet',pooling='avg')


In [30]:
# 讀取 pb 檔
from tensorflow.python.platform import gfile
pb_path = "./overhead_mnist.pb"
graph_def = None
# sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto()) # 嘗試沒用1
with gfile.FastGFile(pb_path, 'rb') as f:
    print("start prepare")
    # graph_def = tf.get_default_graph().as_graph_def()
    # graph_def = tf.GraphDef() # deprecate 換成下面的
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    # sess.graph.as_default() # 嘗試沒用1
    print("read pb fininsh")
# tf.import_graph_def(graph_def, name='')

start prepare
read pb fininsh


c:\Users\Amo\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  # This is added back by InteractiveShellApp.init_path()


In [31]:
# 想要知道 get_tensor_by_name 到底要填入什麼名稱
# 不過不知道為什麼印不出東西來 ?? (現在 read pb 就卡在這裡)
# https://blog.csdn.net/PianGe_zyl/article/details/108155349
with tf.Graph().as_default() as graph :
    tf.import_graph_def(graph_def, name='')
    print(len(graph.get_operations()))
    for op in graph.get_operations():
        print(op.name, op.values())

0


In [ ]:
print(type(graph_def))
print(dir(graph_def))
print(str(graph_def))
print(graph_def._extensions_by_name)

<class 'tensorflow.core.framework.graph_pb2.GraphDef'>
['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'Extensions', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'ParseFromString', 'RegisterExtension', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'UnknownFields', 'WhichOneof', '_CheckCalledFromGeneratedFile', '_SetListener', '__class__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__unicode__', '_extensions_by_name', '_extensions_by_number', '_tf_api_names', '_tf_api_names_v1', 'library', 'node', 'version', 'v

In [ ]:
# These names are part of the model and cannot be changed.
output_layer = 'x:0'
input_node = 'Placeholder:0'

with tf.compat.v1.Session() as sess:
    try:
        prob_tensor = sess.graph.get_tensor_by_name(output_layer)
        predictions = sess.run(prob_tensor, {input_node: [augmented_image] })
    except KeyError:
        print ("Couldn't find classification output layer: " + output_layer + ".")
        print ("Verify this a model exported from an Object Detection project.")
        exit(-1)

Couldn't find classification output layer: x:0.
Verify this a model exported from an Object Detection project.
